In [1]:
# ================================================================================ #
# =========================== Goolge Colab File Upload =========================== #
# ================================================================================ #

from google.colab import drive
drive.mount('/content/drive')

from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/MyDrive/Colab Notebooks/DIYA time series/AttentionLayerExample/Dataset/kor.txt" "kor.txt"
!ls

Mounted at /content/drive
drive  kor.txt	sample_data


In [2]:
import pandas as pd
import re 
import nltk
nltk.download('punkt')

path = "./kor.txt"
df_fra = pd.read_csv(path, names = ["src", "tar", "lic"], sep = "\t")

del df_fra["lic"]
display(df_fra)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,src,tar
0,Go.,가.
1,Hi.,안녕.
2,Run!,뛰어!
3,Run.,뛰어.
4,Who?,누구?
...,...,...
3718,Science fiction has undoubtedly been the inspi...,공상 과학 소설은 의심의 여지 없이 오늘날 존재하는 많은 기술에 영감을 주었어.
3719,I started a new blog. I'll do my best not to b...,난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되지 ...
3720,I think it's a shame that some foreign languag...,몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 있었...
3721,If someone who doesn't know your background sa...,만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 원어...


In [3]:
# library
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
!pip install konlpy
from konlpy.tag import Okt

dataset = df_fra[["src", "tar"]].iloc[:,:]

# preprocessing function

def preprocess_eng(sent):
    
    # 단어 - 구두점 사이의 공간 1 생성
    sent = re.sub(r"([?.!,¿])", r" \1", sent)
    
    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고 전부 공백 전환
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    
    sent = re.sub(r"\s+", " ", sent)
    
    return sent

def preprocess_kor(sent):
    
    sent = word_tokenize(sent)
    
    return sent

test_sent = "Have you ever been to New-York?"
test_sent_pre = preprocess_eng(test_sent)

test_sent_kor = "너는 뉴욕에 가본 적 있니?"
test_sent_kor_pre = preprocess_kor(test_sent_kor)

print(test_sent_pre)
print(test_sent_kor_pre)

     |████████████████████████████████| 19.4MB 297kB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 450kB 55.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Have you ever been to New York ?
['너는', '뉴욕에', '가본', '적', '있니', '?']


In [ ]:
test_sent_kor_pre.append("<sos>")

None
['너는', '뉴욕에', '가본', '적', '있니', '?', '<sos>', '<sos>', '<sos>', '<sos>']


In [4]:
def load_data(df = dataset, data_size = None):
    df_src = df["src"]
    df_tar = df["tar"]
    
    if data_size is None:
        data_size = len(df)
        
    encoder_input, decoder_input, decoder_output = [], [], []
    
    for i in range(1,data_size):
        src_line, tar_line = df_src.iloc[i], df_tar.iloc[i]
        
        # source data preprocessing -> word level separation
        src_line_input = [w for w in preprocess_eng(src_line).split()]
        
        # target data preprocessing -> word level separation
        tar_line = preprocess_kor(tar_line)
        tar_line_input = tar_line.copy()
        tar_line_output = tar_line.copy()

        tar_line_input.insert(0, "<sos>")
        tar_line_output.append("<eos>")

        #tar_line_input = [w for w in ("<sos>" + tar_line).split()]
        #tar_line_output = [w for w in (tar_line + "<eos>").split()]
        
        encoder_input.append(src_line_input)
        decoder_input.append(tar_line_input)
        decoder_output.append(tar_line_output)
        
        del src_line, tar_line, tar_line_input, tar_line_output, src_line_input
    
    return encoder_input, decoder_input, decoder_output


sent_eng_in, sent_kor_in, sent_kor_out = load_data(dataset, 3000)

# word set 

tokenizer_eng = Tokenizer(filters = "", lower = False)
tokenizer_kor = Tokenizer(filters = "", lower = False)

tokenizer_eng.fit_on_texts(sent_eng_in)
encoder_input = tokenizer_eng.texts_to_sequences(sent_eng_in)

tokenizer_kor.fit_on_texts(sent_kor_in)
tokenizer_kor.fit_on_texts(sent_kor_out)

decoder_input = tokenizer_kor.texts_to_sequences(sent_kor_in)
decoder_output = tokenizer_kor.texts_to_sequences(sent_kor_out)

encoder_input = pad_sequences(encoder_input, padding = "post")
decoder_input = pad_sequences(decoder_input, padding = "post")
decoder_output = pad_sequences(decoder_output, padding = "post")

print("encoder_input: ",encoder_input.shape)
print("decoder_input: ",decoder_input.shape)
print("decoder_output: ", decoder_output.shape)

# define word set for src, tar data

src_vocab_size = len(tokenizer_eng.word_index) + 1
tar_vocab_size = len(tokenizer_kor.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 한국어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

src_to_index = tokenizer_eng.word_index
index_to_src = tokenizer_eng.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_kor.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_kor.index_word # 훈련 후 결과 비교할 때 사용

# train test split
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

n_of_val = int(len(indices) * 0.1)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_output_train = decoder_output[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_output_test = decoder_output[-n_of_val:]

encoder_input:  (2999, 11)
decoder_input:  (2999, 11)
decoder_output:  (2999, 11)
영어 단어 집합의 크기 : 2221, 한국어 단어 집합의 크기 : 4195


In [5]:
# model architecture
import tensorflow as tf

# seq2seq model(train)
def seq2seq_model(
    encoder_input_shapes = encoder_input_train.shape[1:],
    decoder_input_shapes = decoder_input_train.shape[1:],
    embed_dim = 64):

    # for test model: translation 과정에서 재호출할 필요가 있는 레이어 집합
    layers_test = []

    # encoder
    encoder_inputs = tf.keras.layers.Input(shape = encoder_input_shapes, name = "encoder_input")
    x_enc = tf.keras.layers.Embedding(src_vocab_size, embed_dim, name = "encoder_embed")(encoder_inputs)
    x_enc = tf.keras.layers.Masking(mask_value = 0.0)(x_enc)
    x_enc, h, c = tf.keras.layers.LSTM(256, return_sequences = True, return_state = True, name = "encoder_lstm")(x_enc)
    enc_states = [h,c]

    layers_test.append(encoder_inputs)
    layers_test.append(enc_states)
    
    # decoder
    decoder_inputs = tf.keras.layers.Input(shape = decoder_input_shapes, name = "decoder_input")
    decoder_embed = tf.keras.layers.Embedding(tar_vocab_size, embed_dim, name = "decoder_embed")
    x_dec = decoder_embed(decoder_inputs)
    x_dec = tf.keras.layers.Masking(mask_value = 0.0)(x_dec)
    
    decoder_lstm = tf.keras.layers.LSTM(256, return_sequences = True, return_state = True, name = "decoder_lstm")
    dec_outputs, _, _ = decoder_lstm(x_dec, initial_state = enc_states)

    decoder_softmax = tf.keras.layers.Dense(tar_vocab_size, activation = "softmax")
    decoder_outputs = decoder_softmax(dec_outputs)
    
    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs, name = "seq2seq")
    model.summary()

    layers_test.append(decoder_inputs)
    layers_test.append(decoder_embed)
    layers_test.append(decoder_lstm)
    layers_test.append(decoder_softmax)
    
    # compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
    loss = 'sparse_categorical_crossentropy'
    metrics = ["acc"]
    model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

    return model, layers_test

# seq2seq model(test)
def test_seq2seq_model(train_model, layers_test, hidden_dims = 256):

    # reload the trained layer
    encoder_inputs = layers_test[0]
    encoder_states = layers_test[1]
    decoder_inputs = layers_test[2]
    decoder_embed = layers_test[3]
    decoder_lstm = layers_test[4]
    decoder_softmax = layers_test[5]

    # encoder model
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states, name = "encoder")

    # decoder model
    decoder_state_input_h = tf.keras.layers.Input(shape = (hidden_dims,))
    decoder_state_input_c = tf.keras.layers.Input(shape = (hidden_dims,))
    decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

    dec_embed2 = decoder_embed(decoder_inputs)

    decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(dec_embed2, initial_state = decoder_state_inputs)

    decoder_states = [decoder_state_h, decoder_state_c]

    decoder_outputs = decoder_softmax(decoder_outputs)

    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states, name = "decoder")

    return encoder_model, decoder_model


# prediction(translate)

def decode_sequence(input_seq, encoder_model, decoder_model):
    state_h = encoder_model.predict(input_seq)

    #<sos> 에 대응되는 정수 인덱스 부여
    target_seq = np.zeros((1,1))
    target_seq[0,0] = tar_to_index['<sos>']
    
    stop_cond = False
    decode_sentence = ''

    while not stop_cond:
        output_tokens, h, c = decoder_model.predict([target_seq] + state_h)

        # 예측 결과를 단어로 변환

        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        if sampled_token_index ==0:
            break
        else:
            sampled_char = index_to_tar[sampled_token_index]

        # 결과를 문장에 추가
        decode_sentence += " " + sampled_char 

        # stop condition
        if (sampled_char == '<eos>' or len(decode_sentence) >= 50):
            stop_cond = True

        # 예측된 결과를 다음 hidden_state로 반영
        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index

        state_h = [h,c]
    
    return decode_sentence

In [6]:
train_model, layers_test = seq2seq_model()

# train model
train_model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_output_train, 
          validation_data = ([encoder_input_test, decoder_input_test], decoder_output_test), epochs = 64,
         batch_size = 64, verbose = 1)

enc, dec = test_seq2seq_model(train_model, layers_test, hidden_dims = 256)

Model: "seq2seq"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 11)]         0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, 11)]         0                                            
__________________________________________________________________________________________________
encoder_embed (Embedding)       (None, 11, 64)       142144      encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_embed (Embedding)       (None, 11, 64)       268480      decoder_input[0][0]              
____________________________________________________________________________________________

In [ ]:
# test
input_sentence = "I love chicken very much~!"
decode_sentence(input_seq, enc, dec)

In [7]:
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!= 0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

for seq_index in [3,10,25,50,100,150,300,500,750,1000,1500, 1750, 2000, 2500]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq, enc, dec)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence)
  print("\n")

원문 :  Who ? 
번역문 : 누구 ? 
예측문 :  누구 ? <eos>


원문 :  Wait ! 
번역문 : 잠깐 ! 
예측문 :  이렇게나 아름다울 수가 ! <eos>


원문 :  Got it ! 
번역문 : 알겠어 ! 
예측문 :  조심해 ! <eos>


원문 :  Hit Tom . 
번역문 : 톰을 때려 . 
예측문 :  톰을 때려 . <eos>


원문 :  Tom paid . 
번역문 : 톰이 지불했어 . 
예측문 :  톰이 입대했어 . <eos>


원문 :  Of course . 
번역문 : 물론이죠 . 
예측문 :  일어나 ! <eos>


원문 :  Come inside . 
번역문 : 안으로 들어와 . 
예측문 :  노래 부탁해 . <eos>


원문 :  I am homesick . 
번역문 : 나 향수병 걸렸어 . 
예측문 :  나 향수병 걸렸어 . <eos>


원문 :  Nobody s perfect . 
번역문 : 완벽한 사람은 없어 . 
예측문 :  그 사람은 고양이를 키워 . <eos>


원문 :  I did nothing wrong . 
번역문 : 난 잘못한 거 없어 . 
예측문 :  난 아주 고립된 기분이었어 . <eos>


원문 :  Tom plays the saxophone . 
번역문 : 톰은 색소폰을 연주할 수 있어 . 
예측문 :  톰은 메리의 감정을 상하게 했다 . <eos>


원문 :  Tom is shouting something . 
번역문 : 톰은 뭔가를 쏘고 있어 . 
예측문 :  톰은 운전할 수 있어 . <eos>


원문 :  I m not sure what they want . 
번역문 : 그들이 원하는 게 뭔지 모르겠다 . 
예측문 :  난 톰이 자폐증이란 걸 알고 있어 . <eos>


원문 :  She has a cat . The cat is white . 
번역문 : 그 사람은 고양이를 키워 . 그 고양이는 흰색 고양이야 . 
예측문 :  그는 나를 기다리게 했다 . <eos>


In [16]:
# attention layer

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # values: hidden states from encoder
        # query: hiden states from decoder, (batch_size, hidden_size)

        ht = tf.expand_dims(query, 1) # (batch_size, 1, hidden_size)
        score = self.V(tf.nn.tanh(self.W1(ht) + self.W2(values)))
        alpha_t = tf.nn.softmax(score, axis = 1) #(batch_size, max_length, 1), attention_weights
        context_vector = alpha_t * values
        context_vector = tf.reduce_sum(context_vector, axis = 1)

        return context_vector, alpha_t


class Encoder(tf.keras.models.Model):
    def __init__(self, embed_dims):
        super(Encoder, self).__init__()
        self.embed = tf.keras.layers.Embedding(src_vocab_size, embed_dims)
        self.lstm = tf.keras.layers.LSTM(256, return_sequences = True, return_state = True)
        self.masking = tf.keras.layers.Masking(mask_value = 0.0)

    def call(self, inputs, h_init = None, c_init = None):
        x = self.embed(inputs)
        x = self.masking(x)
        x, h, c = self.lstm(x, initial_state = [h_init, c_init])
        return x, h, c

class Decoder(tf.keras.models.Model):
    def __init__(self, embed_dims):
        super(Decoder, self).__init__()
        self.embed = tf.keras.layers.Embedding(tar_vocab_size, embed_dims)
        self.lstm = tf.keras.layers.LSTM(256, return_state = True, return_sequences=True)
        self.masking = tf.keras.layers.Masking(mask_value = 0.0)
        self.attention = BahdanauAttention(embed_dims)
        self.softmax = tf.keras.layers.Dense(tar_vocab_size, activation = "softmax")

    def call(self, x, h, c, enc_output):
        
        # 최초에는 t-1번째 hidden_state(decoder) = enc_output
        # x: decoder input
        # h: encoder hidden state(last)
        # c: encoder cell state(last)

        # attention units: embed_dims
        # context_vector: (batch_size, embed_dims)
        # alpha_t : (batch_size, max_len)
        context_vector, alpha_t = self.attention(h, enc_output)

        # embedding: (batch_size, target_max_len, embed_dims)
        x = self.embed(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis = -1)

        x, dec_h, dec_c = self.lstm(x,initial_state = [h,c])
        outputs = self.softmax(x)

        return outputs, dec_h, dec_c


# dataloader with batch size

def data_batch(train_x_enc, train_x_dec, train_y_dec, batch_size = 32):

    # shuffle data
    index = np.arange(0, train_x_enc.shape[0])
    np.random.shuffle(index)

    # batch
    epochs = int(train_x_enc.shape[0] / batch_size)

    for epoch in range(epochs):

        batch_x_enc = train_x_enc[batch_size * epoch : batch_size * (epoch + 1),:]
        batch_x_dec = train_x_dec[batch_size * epoch : batch_size * (epoch + 1),:]

        batch_y_dec = train_y_dec[batch_size * epoch : batch_size * (epoch + 1),:]

        yield batch_x_enc, batch_x_dec, batch_y_dec


# model train
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = "none")

def loss_func(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_obj(real, pred)
    mask = tf.cast(mask, dtype = loss.dtype)
    loss *= mask

    return tf.reduce_mean(loss)

# model build
embed_dims = 64
encoder = Encoder(embed_dims)
decoder = Decoder(embed_dims)

# model save
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder = encoder, decoder=decoder)


# batch 단위 학습을 위한 training function
def train_step(enc_inputs, dec_inputs, dec_outputs, batch_size):

    loss = 0
    enc_h_init = tf.zeros((batch_size, 256))
    enc_c_init = tf.zeros((batch_size, 256))

    with tf.GradientTape() as tape:
        enc_output, enc_state_h, enc_state_c = encoder(enc_inputs, enc_h_init, enc_c_init)
        dec_state_h = enc_state_h
        dec_state_c = enc_state_c


        for t,k in zip(range(0, dec_inputs.shape[1]),range(0, dec_outputs.shape[1])):

            dec_input = tf.expand_dims(dec_inputs[:,t],1)
            dec_output = tf.expand_dims(dec_outputs[:,k],1)

            pred, dec_state_h, dec_state_c = decoder(dec_input, dec_state_h, dec_state_c, enc_output)
            loss += loss_func(dec_output, pred)

    batch_loss = (loss / int(dec_outputs.shape[1]))

    # Backword Propagation and update the weights params
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

EPOCHS = 64
batch_size = 64
epochs_per_steps = int(len(encoder_input_train) / batch_size)

# training
import time

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0
    
    for step in range(epochs_per_steps):

        enc_x_batch = encoder_input_train[batch_size * step: batch_size * (step + 1), :]
        dec_x_batch = decoder_input_train[batch_size * step: batch_size * (step + 1), :]
        dec_y_batch = decoder_output_train[batch_size * step: batch_size * (step + 1), :]

        batch_loss = train_step(enc_x_batch, dec_x_batch, dec_y_batch, batch_size)
        total_loss += batch_loss

        if step % 16 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, step + 1, batch_loss.numpy()))

    # 에포크가 2번 실행될때마다 모델 저장 (체크포인트)
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
        
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / epochs_per_steps))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


Epoch 1 Batch 1 Loss 2.4884
Epoch 1 Batch 17 Loss 2.9436
Epoch 1 Batch 33 Loss 3.4837
Epoch 1 Loss 3.2718
Time taken for 1 epoch 8.312528371810913 sec

Epoch 2 Batch 1 Loss 1.8788
Epoch 2 Batch 17 Loss 2.6905
Epoch 2 Batch 33 Loss 3.1558
Epoch 2 Loss 2.7672
Time taken for 1 epoch 8.080798149108887 sec

Epoch 3 Batch 1 Loss 1.9225
Epoch 3 Batch 17 Loss 2.6976
Epoch 3 Batch 33 Loss 3.0329
Epoch 3 Loss 2.6921
Time taken for 1 epoch 8.085687637329102 sec

Epoch 4 Batch 1 Loss 1.9951
Epoch 4 Batch 17 Loss 2.6172
Epoch 4 Batch 33 Loss 2.9563
Epoch 4 Loss 2.6144
Time taken for 1 epoch 8.07849931716919 sec

Epoch 5 Batch 1 Loss 1.6350
Epoch 5 Batch 17 Loss 2.5296
Epoch 5 Batch 33 Loss 2.9033
Epoch 5 Loss 2.5198
Time taken for 1 epoch 8.11619758605957 sec

Epoch 6 Batch 1 Loss 1.4330
Epoch 6 Batch 17 Loss 2.4492
Epoch 6 Batch 33 Loss 2.8124
Epoch 6 Loss 2.4283
Time taken for 1 epoch 8.09291696548462 sec

Epoch 7 Batch 1 Loss 1.2993
Epoch 7 Batch 17 Loss 2.3909
Epoch 7 Batch 33 Loss 2.7660
Epoch

In [21]:
# prediction(translate)

def decode_sequence(input_seq, encoder_model, decoder_model):

    
    enc_h_init = tf.zeros((1, 256))
    enc_c_init = tf.zeros((1, 256))
    

    # encoder
    enc_output, enc_state_h, enc_state_c = encoder(input_seq, enc_h_init, enc_c_init)

    # decoder
    dec_state_h = enc_state_h
    dec_state_c = enc_state_c

    #<sos>에 대응되는 정수 인덱스 부여
    target_seq = np.zeros((1,1))
    target_seq[0,0] = tar_to_index['<sos>']
    
    stop_cond = False
    decode_sentence = ''

    while not stop_cond:

        output_tokens, dec_state_h, dec_state_c = decoder_model(target_seq, dec_state_h, dec_state_c, enc_output)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        if sampled_token_index ==0:
            break
        else:
            sampled_char = index_to_tar[sampled_token_index]

        # 결과를 문장에 추가
        decode_sentence += " " + sampled_char 

        # stop condition
        if (sampled_char == '<eos>' or len(decode_sentence) >= 50):
            stop_cond = True

        # 예측된 결과를 다음 hidden_state로 반영
        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index
    
    return decode_sentence


def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!= 0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

for seq_index in [3,10,25,50,100,150,300,500,750,1000,1500, 1750, 2000, 2500]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq, encoder, decoder)

    print("원문 : ",seq2src(encoder_input_train[seq_index]))
    print("번역문 :",seq2tar(decoder_input_train[seq_index]))
    print("예측문 :",decoded_sentence)
    print("\n")

원문 :  Who ? 
번역문 : 누구 ? 
예측문 :  누구 집에 있어 ? <eos>


원문 :  Wait ! 
번역문 : 잠깐 ! 
예측문 :  이렇게 매력적일 수가 ! <eos>


원문 :  Got it ! 
번역문 : 알겠어 ! 
예측문 :  이렇게 매력적일 수가 ! <eos>


원문 :  Hit Tom . 
번역문 : 톰을 때려 . 
예측문 :  톰이 외로워하는지 궁금해 . <eos>


원문 :  Tom paid . 
번역문 : 톰이 지불했어 . 
예측문 :  톰이 사기 쳤어 . <eos>


원문 :  Of course . 
번역문 : 물론이죠 . 
예측문 :  빨리 마 . <eos>


원문 :  Come inside . 
번역문 : 안으로 들어와 . 
예측문 :  그만 이것좀 봐 . <eos>


원문 :  I am homesick . 
번역문 : 나 향수병 걸렸어 . 
예측문 :  나는 이유 없이 슬펐어 . <eos>


원문 :  Nobody s perfect . 
번역문 : 완벽한 사람은 없어 . 
예측문 :  아무도 날 알아보지 못 했어 . <eos>


원문 :  I did nothing wrong . 
번역문 : 난 잘못한 거 없어 . 
예측문 :  난 널 도와주려고 노력하던 중이었어 . <eos>


원문 :  Tom plays the saxophone . 
번역문 : 톰은 색소폰을 연주할 수 있어 . 
예측문 :  톰은 나이에 비해 키가 매우 성숙해 . <eos>


원문 :  Tom is shouting something . 
번역문 : 톰은 뭔가를 쏘고 있어 . 
예측문 :  톰은 나이에 비해 키가 매우 성숙해 . <eos>


원문 :  I m not sure what they want . 
번역문 : 그들이 원하는 게 뭔지 모르겠다 . 
예측문 :  난 톰이 웃는 걸 좋아해 . <eos>


원문 :  She has a cat . The cat is white . 
번역문 : 그 사람은 고양이를 키워 . 그 고양이는 